In [ ]:
#import requred library
import requests
import json
import urllib
import os
from getpass import getpass
import time
from datetime import datetime
from msal import ConfidentialClientApplication
import requests
import pandas as pd
from openpyxl import load_workbook

# Get Access token
### Token flow authentication

In [ ]:
URL = 'https://login.microsoftonline.com/common/oauth2/v2.0/authorize'
client_id = "YOUR-CLIENT-ID" # change it with you client ID
permissions = ['files.readwrite']
response_type = 'token'
redirect_uri = 'http://localhost:8080/' 
scope = ''
for items in range(len(permissions)):
    scope = scope + permissions[items]
    if items < len(permissions)-1:
        scope = scope + '+'
        
print('Click over this link ' +URL + '?client_id=' + client_id + '&scope=' + scope + '&response_type=' + response_type+\
     '&redirect_uri=' + urllib.parse.quote(redirect_uri))
print('Sign in to your account, copy the whole redirected URL.')
code = input("Paste the URL here :")
token = code[(code.find('access_token') + len('access_token') + 1) : (code.find('&token_type'))]
URL = 'https://graph.microsoft.com/v1.0/' 
HEADERS = {'Authorization': 'Bearer ' + token}
response = requests.get(URL + 'me/drive/', headers = HEADERS)
if (response.status_code == 200):
    response = json.loads(response.text)
    print('Connected to the OneDrive of', response['owner']['user']['displayName']+' (',response['driveType']+' ).', \
         '\nConnection valid for one hour. Reauthenticate if required.')
elif (response.status_code == 401):
    response = json.loads(response.text)
    print('API Error! : ', response['error']['code'],\
         '\nSee response for more details.')
else: 
    response = json.loads(response.text)
    print('Unknown error! See response for more details.')

## OneDrive operations

In [ ]:
URL = 'https://graph.microsoft.com/v1.0/'

HEADERS = {'Authorization': 'Bearer ' + token}

response = requests.get(URL + 'me/drive/', headers = HEADERS)
if (response.status_code == 200):
    response = json.loads(response.text)
    print('Connected to the OneDrive of', response['owner']['user']['displayName']+' (',response['driveType']+' ).', \
         '\nConnection valid for one hour. Refresh token if required.')
elif (response.status_code == 401):
    response = json.loads(response.text)
    print('API Error! : ', response['error']['code'],\
         '\nSee response for more details.')
else:
    response = json.loads(response.text)
    print('Unknown error! See response for more details.')

### List folders unde root

In [ ]:
items = json.loads(requests.get(URL + 'me/drive/root/children', headers=HEADERS).text)
items = items['value']
for entries in range(len(items)):
    print(items[entries]['name'], '| item-id >', items[entries]['id'])

In [ ]:
## 01BWIL7G557W2SQYZEMFBLSFT5N7MIHCHP this is file ID change it with your file ID where you want to go

In [ ]:
url = URL + 'me/drive/items/01BWIL7G557W2SQYZEMFBLSFT5N7MIHCHP/children'
items = json.loads(requests.get(url, headers=HEADERS).text)
items = items['value']
for entries in range(len(items)):
    print(items[entries]['name'], '| item-id >', items[entries]['id'])

In [ ]:
## reading any excel file using pandas

In [ ]:
url = 'me/drive/root:/Data1.xlsx:/content'
url = URL + url
data = requests.get(url, headers=HEADERS)

In [ ]:
df = pd.read_excel(data.content)

In [ ]:
## reading excel file using workbook

In [ ]:
url = 'me/drive/root:/master.xlsx:/content'
url = URL + url
data = requests.get(url, headers=HEADERS)
with open('master.xlsx', 'wb') as f:
    f.write(data.content)

In [ ]:
wb = load_workbook('master.xlsx')
ws = wb.active

In [ ]:
### performing some operation
# rows = dataframe_to_rows(dfs, index=False, header=False)
# for row in rows:
#     ws.append(row)

In [ ]:
# save the file
wb.save('master.xlsx')

In [ ]:
# upload the file
url = 'me/drive/root:/master.xlsx:/content'
url = URL + url
content = open('master.xlsx', 'rb')
response = json.loads(requests.put(url, headers=HEADERS, data = content).text)